In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch

model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision,
    torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2",
    cache_dir="./moondream2"
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

model = torch.compile(model, mode="max-autotune")


/home/dnth/mambaforge-pypy3/envs/moondream-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
PhiForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the

In [2]:
%%time
image = Image.open('/home/dnth/Desktop/bentoml-model-chaining/phi3.5-vision/btc_chart.jpg')

with torch.inference_mode(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
    enc_image = model.encode_image(image)
    print(model.answer_question(enc_image, "Describe this image.", tokenizer))

The image displays a candlestick chart with a prominent upward trend, overlaid by a candlestick chart with a downward trend, both on a black background.
CPU times: user 770 ms, sys: 158 ms, total: 929 ms
Wall time: 735 ms
